In [6]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import scipy.io
from sklearn.preprocessing import normalize
import random as rnd
from __future__ import division, print_function
from sklearn.metrics import confusion_matrix as cm

mat = scipy.io.loadmat('data5.mat')
data = mat['x']
data = np.asarray(data)
np.random.shuffle(data)
len = data.shape[0]

train = int(0.7 * len)
val = int((0.1 * len) + 1)
test = int((0.2 * len) + 1)
train_data = data[0 : train,:]
val_data = data[train+1 : train+1+val,:]
test_data = data[train+val+1 :,:]

x_train = train_data[:,0:72]
y_train = train_data[:,-1]
x_val = val_data[:,0:72]
y_val = val_data[:,-1]
x_test = test_data[:,0:72]
y_test = test_data[:,-1]

for i in range(y_train.shape[0]):
  y_train[i] = int(y_train[i])
  if int(y_train[i]) == 0:
    y_train[i] = -1
for i in range(y_val.shape[0]):
  y_val[i] = int(y_val[i])
  if int(y_val[i]) == 0:
    y_val[i] = -1
for i in range(y_test.shape[0]):
  y_test[i] = int(y_test[i])
  if int(y_test[i]) == 0:
    y_test[i] = -1

class SVM():
    def __init__(self, max_iter=5000, kernel_type='linear', C=1.0, epsilon=0.001):
        self.kernels = {
            'linear' : self.kernel_linear,
            'quadratic' : self.kernel_quadratic
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon

    def fit(self, X, y):
        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernels[self.kernel_type]
        count = 0
        while True:
            count += 1
            alpha_prev = np.copy(alpha)
            for j in range(0, n):
                i = self.get_rnd_int(0, n-1, j) 
                x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if k_ij == 0:
                    continue
                alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
                (L, H) = self.compute_L_H(self.C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                self.w = self.calc_w(alpha, y, X)
                self.b = self.calc_b(X, y, self.w)

                E_i = self.E(x_i, y_i, self.w, self.b)
                E_j = self.E(x_j, y_j, self.w, self.b)

                alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break

            if count >= self.max_iter:
                return

        self.b = self.calc_b(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.calc_w(alpha, y, X)

        alpha_ind = np.where(alpha > 0)[0]
        support_vectors = X[alpha_ind, :]
        return support_vectors, count

    def predict(self, X):
        return self.h(X, self.w, self.b)

    def calc_b(self, X, y, w):
        b_tmp = y - np.dot(w.T, X.T)
        return np.mean(b_tmp)

    def calc_w(self, alpha, y, X):
        return np.dot(X.T, np.multiply(alpha,y))

    def h(self, X, w, b):
        return np.sign(np.dot(w.T, X.T) + b).astype(int)

    def E(self, x_k, y_k, w, b):
        return self.h(x_k, w, b) - y_k

    def compute_L_H(self, C, alpha_prime_j, alpha_prime_i, yj, yi):
        if(yi != yj):
            return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
        else:
            return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))

    def get_rnd_int(self, a,b,z):
        i = z
        count=0
        while i == z and count<1000:
            i = rnd.randint(a,b)
            count += 1
        return i

    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)

    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)

def print_acc(y_pred,y_actual):
    confmat = cm(y_actual, y_pred)
    Accuracy = (confmat[0][0]+confmat[1][1])/ (np.shape(y_pred)[0])
    se = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    sp = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    
    print(confmat)
    print('\nAccuracy: ',Accuracy,'sensitivity: ',se,'specificity: ',sp)

svm = SVM()
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)

print_acc(y_pred,y_test)

#Best accuracy recorded so far: 0.901392...

[[157  54]
 [ 21 197]]

Accuracy:  0.8251748251748252 sensitivity:  0.9036697247706422 specificity:  0.7440758293838863
